# Working with Images, Part 1


# Univeral Code Used for the Entire Notebook

Let's set up our libraries and client

In [ ]:
# Import the OpenAI class from the openai module
import requests
import base64
from openai import OpenAI
from IPython.display import Image, display, HTML

In [ ]:
# Create a client we can use
client = OpenAI()

## Generations

The image generations endpoint allows you to create an original image given a text prompt. When using DALL·E 3, images can have a size of 1024x1024, 1024x1792 or 1792x1024 pixels.

By default, images are generated at standard quality, but when using DALL·E 3 you can set quality: "hd" for enhanced detail. Square, standard quality images are the fastest to generate.

You can request 1 image at a time with DALL·E 3 (request more by making parallel requests) or up to 10 images at a time using DALL·E 2 with the n parameter.

In [ ]:
# Simple image generation
image = client.images.generate(
    prompt="a cabin on a snowy hillside at night",
)

image_url = image.data[0].url

# Display the image
print(f"Image URL: {image_url}")
display(Image(url=image_url))

In [ ]:
# Full parameter generation
image_full_parameters = client.images.generate(
    prompt="a cabin on a snowy hillside at night",
    model="dall-e-3",
    n=1,
    quality="standard",
    response_format="url",
    size="1024x1024",
    style="natural",
    user="user-id-123"
)

image_url = image_full_parameters.data[0].url

# Display the image
print(f"Image URL: {image_url}")
display(Image(url=image_url))

In [ ]:
# Generate multiple images
images = client.images.generate(
    prompt="a cabin on a snowy hillside at night",
    model="dall-e-2",
    n=10,
    quality="standard",
    response_format="url",
    size="256x256",
    style="natural",
    user="user-id-123"
)

# Extract URLs from the responses and display them
html = ""
for image in images.data:
    image_url = image.url  # Access the URL from the Image object
    html += f'<img src="{image_url}" style="display:inline; margin:10px; width:256px; height:256px;">'
    print(f"{image_url}\n")

# Display the images side by side
display(HTML(html))


## Examining the Revised Prompt


In [ ]:
# Full parameter generation
prompt = "a cabin on a snowy hillside at night"

image_full_parameters = client.images.generate(
    prompt="a cabin on a snowy hillside at night",
    model="dall-e-3",
    n=1,
    quality="standard",
    response_format="url",
    size="1024x1024",
    style="natural",
    user="user-id-123"
)

image_url = image_full_parameters.data[0].url

# Show the before and after prompts
print("Original Prompt:")
print(prompt)
print("\n")
print("Revised Prompt:")
print(image_full_parameters.data[0].revised_prompt)


# Display the image
print(f"\nImage URL: {image_url}")
display(Image(url=image_url))

### Overriding the Revised Prompt (Doesn't Work)

In [ ]:
# Full parameter generation
prompt = "I NEED to test how the tool works with extremely simple prompts. DO NOT add any detail, just use it AS-IS: a cabin on a snowy hillside at night"

image_full_parameters = client.images.generate(
    prompt="a cabin on a snowy hillside at night",
    model="dall-e-3",
    n=1,
    quality="standard",
    response_format="url",
    size="1024x1024",
    style="natural",
    user="user-id-123"
)

image_url = image_full_parameters.data[0].url

# Show the before and after prompts
print("Original Prompt:")
print(prompt)
print("\n")
print("Revised Prompt:")
print(image_full_parameters.data[0].revised_prompt)


# Display the image
print(f"\nImage URL: {image_url}")
display(Image(url=image_url))

### Download Image from URL


In [ ]:
# Full parameter generation
image_full_parameters = client.images.generate(
    prompt="a cabin on a snowy hillside at night",
    model="dall-e-3",
    n=1,
    quality="standard",
    response_format="url",
    size="1024x1024",
    style="natural",
    user="user-id-123"
)

image_url = image_full_parameters.data[0].url

# Display the image
display(Image(url=image_url))

# Download the image
image_data = requests.get(image_url).content

# Save the image to a file
with open('snowy_cabin.png', 'wb') as handler:
    handler.write(image_data)

print("Image downloaded and saved as 'snowy_cabin.png'")

## Return and Download a Base64 image instead of a URL

URL's expire after an hour so, you either need to download them before that time limit or, if you want the image to persist, you should create a Base64 image. 



In [ ]:
# Full parameter generation
image_full_parameters = client.images.generate(
    prompt="a cabin on a snowy hillside at night",
    model="dall-e-3",
    n=1,
    quality="standard",
    response_format="b64_json",
    size="1024x1024",
    style="natural",
    user="user-id-123"
)

# Get the Base64 image data
b64_image = image_full_parameters.data[0].b64_json

# Decode the Base64 image
image_data = base64.b64decode(b64_image)

# Display the image
display(Image(data=image_data))

# Optionally save the image to a file
with open('b64_snowy_cabin.png', 'wb') as handler:
    handler.write(image_data)

print("Image displayed and saved as 'b64_snowy_cabin.png'")

# Working with Images, Part 2

## Editing Images (DALL-E 2 Only)

In [ ]:
# Full parameter generation
image_full_parameters = client.images.generate(
    prompt="a pool with a nice house in the background",
    model="dall-e-2",
    n=1,
    quality="standard",
    response_format="url",
    size="1024x1024",
    style="natural",
    user="user-id-123"
)

image_url = image_full_parameters.data[0].url

# Display the image
display(Image(url=image_url))

# Download the image
image_data = requests.get(image_url).content

# Save the image to a file
with open('pool_original.png', 'wb') as handler:
    handler.write(image_data)

print("Image downloaded and saved as 'pool_original.png'")

In [ ]:
from PIL import Image
import openai

# Open the original image
original_image = Image.open("pool_original.png")

# Convert the image to RGBA format
rgba_image = original_image.convert("RGBA")

# Save the converted image to a new file
rgba_image.save("pool_rgba.png")



In [5]:
# Now use the converted image in your API request
response = client.images.edit(
    image=open("pool_rgba.png", "rb"),
    mask=open("pool_rgba_mask.png", "rb"),
    prompt="add a flamingo toy to the water",
    n=1,
    size="1024x1024"
)

# Get the URL of the generated image
image_url = response['data'][0]['url']

# Display the image
from IPython.display import Image as IPImage, display
display(IPImage(url=image_url))

## Variations (DALL-E 2 Only)

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.images.create_variation(
    model="dall-e-2",
    image=open("pool_rgba.png", "rb"),
    n=1,
    size="1024x1024"
)

image_url = response.data[0].url

# Display the image
display(Image(url=image_url))